📌 1. Imports

In [ ]:
# Regular expressions for parsing input lines
import re

# Dictionary with default values for nested transitions
from collections import defaultdict

# For drawing the DFA using Graphviz
import graphviz

# For uploading files in Google Colab
from google.colab import files

📌 2. DFA Class Definition and Initialization

In [ ]:
class DFA:
    def __init__(self, alphabet, states, initial, finals, transitions):
        # Basic DFA components
        self.alphabet = alphabet
        self.states = states
        self.initial = initial
        self.finals = finals
        self.transitions = transitions


📌 3. Loading DFA from a .txt File

In [ ]:
    @staticmethod
    def from_txt(file_path):
        # Load and parse the DFA definition from a formatted text file
        with open(file_path, 'r') as f:
            lines = [line.strip() for line in f if line.strip() and not line.startswith('#')]

        alphabet = states = finals = transitions = None
        initial = None
        trans = []

        for line in lines:
            if line.startswith('alfabeto:'):
                alphabet = line.split(':')[1].split(',')
            elif line.startswith('estados:'):
                states = line.split(':')[1].split(',')
            elif line.startswith('inicial:'):
                initial = line.split(':')[1]
            elif line.startswith('finais:'):
                finals = line.split(':')[1].split(',')
            elif re.match(r'^\w+,\w+,\w+$', line):
                trans.append(tuple(line.split(',')))

        if not all([alphabet, states, initial, finals, trans]):
            raise ValueError("Input file is incomplete or malformed.")

        transitions = defaultdict(dict)
        for origin, dest, symbol in trans:
            if symbol in transitions[origin]:
                raise ValueError(f"Nondeterministic transition from {origin} on symbol {symbol}.")
            transitions[origin][symbol] = dest

        dfa = DFA(alphabet, states, initial, finals, transitions)
        dfa.validate()
        return dfa


📌 4. Validation Method

In [ ]:
    def validate(self):
        # Ensure all states and symbols are valid
        if self.initial not in self.states:
            raise ValueError("Initial state is not in the list of states.")
        for f in self.finals:
            if f not in self.states:
                raise ValueError(f"Final state {f} is not valid.")
        for origin, trans in self.transitions.items():
            if origin not in self.states:
                raise ValueError(f"Invalid origin state: {origin}")
            for symbol, dest in trans.items():
                if symbol not in self.alphabet:
                    raise ValueError(f"Invalid symbol in transition: {symbol}")
                if dest not in self.states:
                    raise ValueError(f"Invalid destination state: {dest}")
